# UB File Splitter

##### Instructions:

1. Enter the route.
2. Enter the commit.
3. Upload the file to process.
4. Click the "Process File" button.<br></br>

In [142]:
from collections import Counter 
import os
import pdfplumber
from PyPDF2 import PdfReader, PdfWriter
import ipywidgets as widgets

In [144]:
output = widgets.Output()

def create_text_input(description, value):
    return widgets.Text(description=description, value=value)

form_item_layout = widgets.Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between',
    height='50px'
)

route_input = create_text_input('Route:', '')
commit_input = create_text_input('Commit:', '')
file_input = widgets.FileUpload(accept='.pdf', multiple=False, layout=widgets.Layout())

submit_button = widgets.Button(
    description='Process Files',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Process Files',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

form_items = [
    widgets.Box([route_input, commit_input], layout=form_item_layout),
    widgets.Box([widgets.Label(value='Munis File Upload:'), file_input], layout=form_item_layout),
    widgets.Box([submit_button], layout=form_item_layout)
]

form = widgets.Box(form_items, layout=widgets.Layout(
    display='flex',
    flex_flow='column',
    # border='solid 2px',
    align_items='center',
    width='100%',
))




In [145]:


multi_bill_page_numbers = []
single_bill_page_num_buffer = []

def on_button_clicked(b):
    with output:
        single_page_filename = f"Route_{route_input.value}-{commit_input.value}-single-page-bills.pdf"
        multi_page_filename = f"Route_{route_input.value}-{commit_input.value}-multi-page-bills.pdf"

        pdf_file_path = file_input.value[0]
        single_page_filepath = os.path.join('/', single_page_filename)
        multi_page_filepath = os.path.join('/', multi_page_filename)
        

        print(output)
        print(route_input.value)
        
        with pdfplumber.open(pdf_file_path) as pdf:
            total_pages = len(pdf.pages)
            print(f"Total Pages: {total_pages}")
            previous_bill_number = ''

            for i in range(1, total_pages+1, 1):
                page = pdf.pages[i-1]
                page_text = page.extract_text().split('\n')
                bounding_box = (249.1, 50.8887, 306.7, 69.8)
                crop_area = page.crop(bounding_box)
                crop_text = crop_area.extract_text().split('\n')

                if crop_text == previous_bill_number:
                    multi_bill_page_numbers.append(i-1)
                    multi_bill_page_numbers.append(i)
                else:
                    single_bill_page_num_buffer.append(i)

                previous_bill_number = crop_text


        single_bill_page_numbers = list((Counter(single_bill_page_num_buffer) - Counter(multi_bill_page_numbers)).elements())

        pdf = PdfReader(pdf_file_path)
        pdfWriter = PdfWriter()
        
        print(f"List of Multi-Page Bill Pages: {multi_bill_page_numbers}")

        for multi_page_num in multi_bill_page_numbers:
            pdfWriter.add_page(pdf.pages[multi_page_num-1])

            with open(multi_page_filepath, 'wb') as f:
                pdfWriter.write(f)
                f.close()


        print(f"List of Single Bill Pages: {single_bill_page_numbers}")

        pdf = PdfReader(pdf_file_path)
        pdfWriter = PdfWriter()

        for single_page_num in single_bill_page_numbers:
            pdfWriter.add_page(pdf.pages[single_page_num-1])

            with open(single_page_filepath, 'wb') as f:
                pdfWriter.write(f)
                f.close()

        print("Go to Output Files to Download the Processed Bill Files")
        
        
        
        
        
        
        
display(form, output)
submit_button.on_click(on_button_clicked)

Box(children=(Box(children=(Text(value='', description='Route:'), Text(value='', description='Commit:')), layo…

Output()